In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clip_captioner/pytorch/default/2/torch_model (2).pt


In [2]:
!pip install pandas torch clip

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6988 sha256=d328df6d986c4be987a96d0f99f4fd9d9ade211dbadcc33d67f72b9771d9292a
  Stored in directory: /root/.cache/pip/wheels/ab/a5/e8/c9fa20742edbccf2702dae8ee62053e6c460e961d45967b49c
Successfully built clip
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstallin

# Load Dataset 

In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("Yasintuncer/NIH-CXR14-BiomedCLIP-Features")
df = pd.DataFrame(dataset['train'])


README.md:   0%|          | 0.00/5.60k [00:00<?, ?B/s]

features_0.parquet:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

features_1.parquet:   0%|          | 0.00/55.4M [00:00<?, ?B/s]

features_2.parquet:   0%|          | 0.00/58.9M [00:00<?, ?B/s]

features_3.parquet:   0%|          | 0.00/62.9M [00:00<?, ?B/s]

features_4.parquet:   0%|          | 0.00/61.2M [00:00<?, ?B/s]

features_5.parquet:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112120 [00:00<?, ? examples/s]

# Split Dataset 

In [4]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(
    df,
    test_size=0.20,
    random_state=42,     
    shuffle=True          
)

train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.25,      
    random_state=42,
    shuffle=True
)

print(f"Train: {len(train_df)} rows")
print(f"Val:   {len(val_df)} rows")
print(f"Test:  {len(test_df)} rows")
from torch.utils.data import DataLoader

train_loader = DataLoader(train_df, batch_size=16, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_df,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_df,  batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

Train: 67272 rows
Val:   22424 rows
Test:  22424 rows


# Inspect Vocabulary 

In [5]:
import string
lower_text = df["Texts"].apply(lambda x: x.lower())
texts = lower_text.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
doc_lengths = texts.apply(lambda x: len(x.split()))
doc_lengths.describe() 

count    112120.000000
mean         18.633330
std           1.362298
min          18.000000
25%          18.000000
50%          18.000000
75%          18.000000
max          28.000000
Name: Texts, dtype: float64

In [6]:
from collections import Counter

# Build vocabulary
vocab = Counter()
for doc in texts:
    vocab.update(doc.lower().split())

print(vocab)

print(f"Total vocabulary size: {len(vocab)}")
print(f"Average vocabulary size per document: {doc_lengths.mean():.2f}")



Counter({'a': 255203, 'this': 112120, 'photo': 112120, 'of': 112120, 'chest': 112120, 'xray': 112120, 'shows': 112120, 'the': 112120, 'image': 112120, 'is': 112120, 'taken': 112120, 'from': 112120, 'view': 112120, 'findings': 81157, 'pa': 67310, 'no': 60361, 'available': 60361, 'ap': 44810, 'finding': 30963, 'multiple': 20796, 'including': 20796, 'and': 20796, 'infiltration': 19894, 'effusion': 13317, 'atelectasis': 11559, 'nodule': 6331, 'mass': 5782, 'pneumothorax': 5302, 'consolidation': 4667, 'pleuralthickening': 3385, 'cardiomegaly': 2776, 'emphysema': 2516, 'edema': 2303, 'fibrosis': 1686, 'pneumonia': 1431, 'hernia': 227})
Total vocabulary size: 36
Average vocabulary size per document: 18.63


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67272 entries, 32166 to 86485
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Image Index         67272 non-null  object
 1   Texts               67272 non-null  object
 2   View Position       67272 non-null  object
 3   Image Features      67272 non-null  object
 4   Text Features       67272 non-null  object
 5   Atelectasis         67272 non-null  int64 
 6   Cardiomegaly        67272 non-null  int64 
 7   Effusion            67272 non-null  int64 
 8   Infiltration        67272 non-null  int64 
 9   Mass                67272 non-null  int64 
 10  Nodule              67272 non-null  int64 
 11  Pneumonia           67272 non-null  int64 
 12  Pneumothorax        67272 non-null  int64 
 13  Consolidation       67272 non-null  int64 
 14  Edema               67272 non-null  int64 
 15  Emphysema           67272 non-null  int64 
 16  Fibrosis            672

# Training GPT-2 Model 
Since we have a dataset composed of mainly clip image embeddings and text outputs associated to them we have decided to train a Decoder Only Transformer Model that is capable of using the embeddings as an input and generate captions for the embeddings in an autoregressive way.

We have taken the following code from: 
- [Clip Caption - GitHub](https://github.com/rmokady/CLIP_prefix_caption/tree/main?tab=readme-ov-file) 
- [Clip Caption - Arxiv](https://arxiv.org/abs/2111.09734)
## Format data for `CLIP_prefix_caption`

In [8]:
EOS_TOKEN = '<|endoftext|>'

In [9]:
import torch
import pickle
all_embeddings = []
caption_dicts  = []

train_df = train_df.sample(64000) ## 

for i, (_, row) in enumerate(train_df.iterrows()):
    all_embeddings.append(torch.as_tensor(row["Image Features"], dtype=torch.float32))
    caption_dicts.append({
        "caption":        row["Texts"] + EOS_TOKEN ,
        "image_id":       row["Image Index"],
        "clip_embedding": i,     
    })

   

embeddings_tensor = torch.stack(all_embeddings, dim=0)

with open("./train_data1.pkl", "wb") as f:
    pickle.dump({
        "clip_embedding": embeddings_tensor,
        "captions":       caption_dicts,
    }, f)

## Define the models 


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from enum import Enum
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union


class MappingType(Enum):
    MLP = 'mlp'
    Transformer = 'transformer'


class ClipCocoDataset(Dataset):

    def __len__(self) -> int:
        return len(self.captions_tokens)

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.captions_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask

    def __getitem__(self, item: int) -> Tuple[torch.Tensor, ...]:
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[self.caption2embedding[item]]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

    def __init__(self, data_path: str,  prefix_length: int, gpt2_type: str = "gpt2",
                 normalize_prefix=False):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)
        print("Data size is %0d" % len(all_data["clip_embedding"]))
        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        self.captions = [caption['caption'] for caption in captions_raw]
        if os.path.isfile(f"{data_path[:-4]}_tokens.pkl"):
            with open(f"{data_path[:-4]}_tokens.pkl", 'rb') as f:
                self.captions_tokens, self.caption2embedding, self.max_seq_len = pickle.load(f)
        else:
            self.captions_tokens = []
            self.caption2embedding = []
            max_seq_len = 0
            for caption in captions_raw:
                self.captions_tokens.append(torch.tensor(self.tokenizer.encode(caption['caption']), dtype=torch.int64))
                self.caption2embedding.append(caption["clip_embedding"])
                max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])
            # self.max_seq_len = max_seq_len
            with open(f"{data_path[:-4]}_tokens.pkl", 'wb') as f:
                pickle.dump([self.captions_tokens, self.caption2embedding, max_seq_len], f)
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))


class MLP(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class MlpTransformer(nn.Module):
    def __init__(self, in_dim, h_dim, out_d: Optional[int] = None, act=nnf.relu, dropout=0.):
        super().__init__()
        out_d = out_d if out_d is not None else in_dim
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.act = act
        self.fc2 = nn.Linear(h_dim, out_d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class MultiHeadAttention(nn.Module):

    def __init__(self, dim_self, dim_ref, num_heads, bias=True, dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim_self // num_heads
        self.scale = head_dim ** -0.5
        self.to_queries = nn.Linear(dim_self, dim_self, bias=bias)
        self.to_keys_values = nn.Linear(dim_ref, dim_self * 2, bias=bias)
        self.project = nn.Linear(dim_self, dim_self)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y=None, mask=None):
        y = y if y is not None else x
        b, n, c = x.shape
        _, m, d = y.shape
        # b n h dh
        queries = self.to_queries(x).reshape(b, n, self.num_heads, c // self.num_heads)
        # b m 2 h dh
        keys_values = self.to_keys_values(y).reshape(b, m, 2, self.num_heads, c // self.num_heads)
        keys, values = keys_values[:, :, 0], keys_values[:, :, 1]
        attention = torch.einsum('bnhd,bmhd->bnmh', queries, keys) * self.scale
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(1)
            attention = attention.masked_fill(mask.unsqueeze(3), float("-inf"))
        attention = attention.softmax(dim=2)
        out = torch.einsum('bnmh,bmhd->bnhd', attention, values).reshape(b, n, c)
        out = self.project(out)
        return out, attention


class TransformerLayer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        x_, attention = self.attn(self.norm1(x), y, mask)
        x = x + x_
        x = x + self.mlp(self.norm2(x))
        return x, attention

    def forward(self, x, y=None, mask=None):
        x = x + self.attn(self.norm1(x), y, mask)[0]
        x = x + self.mlp(self.norm2(x))
        return x

    def __init__(self, dim_self, dim_ref, num_heads, mlp_ratio=4., bias=False, dropout=0., act=nnf.relu,
                 norm_layer: nn.Module = nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim_self)
        self.attn = MultiHeadAttention(dim_self, dim_ref, num_heads, bias=bias, dropout=dropout)
        self.norm2 = norm_layer(dim_self)
        self.mlp = MlpTransformer(dim_self, int(dim_self * mlp_ratio), act=act, dropout=dropout)


class Transformer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        attentions = []
        for layer in self.layers:
            x, att = layer.forward_with_attention(x, y, mask)
            attentions.append(att)
        return x, attentions

    def forward(self, x, y=None, mask=None):
        for i, layer in enumerate(self.layers):
            if i % 2 == 0 and self.enc_dec: # cross
                x = layer(x, y)
            elif self.enc_dec:  # self
                x = layer(x, x, mask)
            else:  # self or cross
                x = layer(x, y, mask)
        return x

    def __init__(self, dim_self: int, num_heads: int, num_layers: int, dim_ref: Optional[int] = None,
                 mlp_ratio: float = 2., act=nnf.relu, norm_layer: nn.Module = nn.LayerNorm, enc_dec: bool = False):
        super(Transformer, self).__init__()
        dim_ref = dim_ref if dim_ref is not None else dim_self
        self.enc_dec = enc_dec
        if enc_dec:
            num_layers = num_layers * 2
        layers = []
        for i in range(num_layers):
            if i % 2 == 0 and enc_dec:  # cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            elif enc_dec:  # self
                layers.append(TransformerLayer(dim_self, dim_self, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            else:  # self or cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
        self.layers = nn.ModuleList(layers)


class TransformerMapper(nn.Module):

    def forward(self, x):
        x = self.linear(x).view(x.shape[0], self.clip_length, -1)
        prefix = self.prefix_const.unsqueeze(0).expand(x.shape[0], *self.prefix_const.shape)
        prefix = torch.cat((x, prefix), dim=1)
        out = self.transformer(prefix)[:, self.clip_length:]
        return out

    def __init__(self, dim_clip: int, dim_embedding: int, prefix_length: int, clip_length: int, num_layers: int = 8):
        super(TransformerMapper, self).__init__()
        self.clip_length = clip_length
        self.transformer = Transformer(dim_embedding, 8, num_layers)
        self.linear = nn.Linear(dim_clip, clip_length * dim_embedding)
        self.prefix_const = nn.Parameter(torch.randn(prefix_length, dim_embedding), requires_grad=True)


class ClipCaptionModelTrain(nn.Module):

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, clip_length: Optional[int] = None, prefix_size: int = 512,
                 num_layers: int = 8, mapping_type: MappingType = MappingType.MLP):
        super(ClipCaptionModelTrain, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if mapping_type == MappingType.MLP:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                     self.gpt_embedding_size * prefix_length))
        else:
            self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, prefix_length,
                                                                     clip_length, num_layers)


class ClipCaptionPrefix(ClipCaptionModelTrain):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self


def save_config(args: argparse.Namespace):
    config = {}
    for key, item in args._get_kwargs():
        config[key] = item
    out_path = os.path.join(args.out_dir, f"{args.prefix}.json")
    with open(out_path, 'w') as outfile:
        json.dump(config, outfile)


def load_model(config_path: str, epoch_or_latest: Union[str, int] = '_latest'):
    with open(config_path) as f:
        config = json.load(f)
    parser = argparse.ArgumentParser()
    parser.set_defaults(**config)
    args = parser.parse_args()
    if type(epoch_or_latest) is int:
        epoch_or_latest = f"-{epoch_or_latest:03d}"
    model_path = os.path.join(args.out_dir, f"{args.prefix}{epoch_or_latest}.pt")
    if args.only_prefix:
        model = ClipCaptionPrefix(args.prefix_length)
    else:
        model = ClipCaptionModelTrain(args.prefix_length)
    if os.path.isfile(model_path):
        print(f"loading model from {model_path}")
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    else:
        print(f"{model_path} is not exist")
    return model, parser


def train(dataset: ClipCocoDataset, model: ClipCaptionModelTrain, args,
          lr: float = 2e-5, warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):

    device = torch.device('cuda:0')
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = model.to(device)
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    num_training_steps = epochs * len(train_dataloader)
    scheduler = get_scheduler(
        name="linear",            # linear warmup & decay
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=num_training_steps,
    )
    # save_config(args)
    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]
            loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            progress.set_postfix({"loss": loss.item()})
            progress.update()
            if (idx + 1) % 10000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        progress.close()
        if epoch % args.save_every == 0 or epoch == epochs - 1:
            torch.save(
                model.state_dict(),
                "./torch_model.pt",
            )
    return model


def run_train(
    data: str = './train_data1.pkl',
    out_dir: str = './checkpoints',
    prefix: str = 'coco_prefix',
    epochs: int = 3,
    save_every: int = 1,
    prefix_length: int = 10,
    prefix_length_clip: int = 10,
    bs: int = 40,
    only_prefix: bool = False,
    mapping_type: str = 'mlp',
    num_layers: int = 8,
    is_rn: bool = False,
    normalize_prefix: bool = False,
):
    dataset = ClipCocoDataset(data, prefix_length, normalize_prefix=normalize_prefix)

    prefix_dim = 512

    # 3) Resolve mapping_type enum
    mapping_enum = {
        'mlp': MappingType.MLP,
        'transformer': MappingType.Transformer
    }[mapping_type]

    if only_prefix:
        model = ClipCaptionPrefix(
            prefix_length,
            clip_length=prefix_length_clip,
            prefix_size=prefix_dim,
            num_layers=num_layers,
            mapping_type=mapping_enum
        )
        print("Train only prefix")
    else:
        model = ClipCaptionModelTrain(
            prefix_length,
            clip_length=prefix_length_clip,
            prefix_size=prefix_dim,
            num_layers=num_layers,
            mapping_type=mapping_enum
        )
        print("Train both prefix and GPT")

    model = train(
        dataset,
        model,
        args=type('X', (), {
            'epochs': epochs,
            'save_every': save_every,
            'bs': bs
        })(),
        lr=2e-5,
        warmup_steps=5000,
        output_dir=out_dir,
        output_prefix=prefix
    )

    return model 


#trained_model = run_train()

2025-05-18 21:04:15.517701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747602255.740871      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747602255.802561      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Inference 


In [11]:
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    get_linear_schedule_with_warmup,
)
import skimage.io as io
import PIL.Image


N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]


D = torch.device



class MLP(nn.Module):
    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):

    # @functools.lru_cache #FIXME
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(
            batch_size, self.prefix_length, dtype=torch.int64, device=device
        )

    def forward(
        self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None
    ):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(
            -1, self.prefix_length, self.gpt_embedding_size
        )
        # print(embedding_text.size()) #torch.Size([5, 67, 768])
        # print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 512):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained("gpt2")
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(
                prefix_size, self.gpt_embedding_size * prefix_length
            )
        else:
            self.clip_project = MLP(
                (
                    prefix_size,
                    (self.gpt_embedding_size * prefix_length) // 2,
                    self.gpt_embedding_size * prefix_length,
                )
            )


class ClipCaptionPrefix(ClipCaptionModel):
    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self


def generate_beam(
    model,
    tokenizer,
    beam_size: int = 5,
    prompt=None,
    embed=None,
    entry_length=67,
    temperature=1.0,
    stop_token: str = ".",
):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(
                    beam_size, -1
                )
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(
                generated.shape[0], 1, -1
            )
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [
        tokenizer.decode(output[: int(length)])
        for output, length in zip(output_list, seq_lengths)
    ]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
    model,
    tokenizer,
    tokens=None,
    prompt=None,
    embed=None,
    entry_count=1,
    entry_length=67,  # maximum number of words
    top_p=0.8,
    temperature=1.0,
    stop_token: str = ".",
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in range(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(
                    nnf.softmax(sorted_logits, dim=-1), dim=-1
                )
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]

### Run the model 
- if you want to run the model from the weights just replace MODEL_PATH with the path to the weights else leave it empty
- if you want to run the model using beam search just change the `use_beam_search` var to `True`
  

## Defining Benchmarks 
Since our dataset is a dataset with Image Classification  we will use Accuracy/f1/precision and recall as benchmark metrics and for the text generation part we will use ROUGE to calculate the overlapping of the Reference Text with the Generated Texts. 

Since our dataset is big and our free available compute power is limited we will run the benchmarks for 100 samples. 

In [12]:
MODEL_PATH = "/kaggle/input/clip_captioner/pytorch/default/2/torch_model (2).pt" ## use this if you have the .pt file 
#MODEL_PATH = ""
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PREFIX_LENGTH = 10

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# load the fine tuned model
if MODEL_PATH != "": 
    model = ClipCaptionModel(PREFIX_LENGTH)
    state_dict = torch.load(MODEL_PATH, map_location="cpu")
    model.load_state_dict(state_dict)
    model.eval().to(DEVICE)
else: 
    model = trained_model 


def generate_caption(
    model,
    clip_embedding,
    use_beam_search = False,
    beam_size = 5,
    top_p = 0.8,
    temperature = 1.0,
    max_length = 67
):
    clip_embedding = clip_embedding.to(DEVICE).to(torch.float32)
    prefix_embed = model.clip_project(clip_embedding)
    prefix_embed = prefix_embed.view(1, PREFIX_LENGTH, -1)

    if use_beam_search:
        captions = generate_beam(
            model,
            tokenizer,
            beam_size=beam_size,
            embed=prefix_embed,
            entry_length=max_length,
            stop_token = EOS_TOKEN,
            temperature=temperature,
        )
        return captions[0]
    else:
        return generate2(
            model,
            tokenizer,
            embed=prefix_embed,
            entry_length=max_length,
            top_p=top_p,
            stop_token = EOS_TOKEN,
            temperature=temperature,
        )


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/tmp/ipykernel_19/3755249001.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location="cpu")


In [13]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=db89a7e7a5eb55446e733002f01705abac5f63ecc14ddbacc3d43625635de81b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 w

## Generate Outputs for Test DS items 

In [14]:
rnd_state = 42
test_samples_100 = test_df.sample(n=100,random_state=rnd_state)

base_text = []
pred_text = []

for _ , row in test_samples_100.iterrows(): 
    base_text.append(row["Texts"])
    pred_text.append(generate_caption(model,torch.FloatTensor(row["Image Features"])).replace(EOS_TOKEN, ""))


In [15]:
import evaluate 
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score 

rouge = evaluate.load('rouge')
label_cols = [
"Atelectasis", "Cardiomegaly", "Effusion", "Infiltration",
"Mass", "Nodule", "Pneumonia", "Pneumothorax",
"Consolidation", "Edema", "Emphysema", "Fibrosis",
"Hernia", "Pleural Thickening", "no findings","AP", "PA"
]

def eval_model(pred_texts, base_texts):
    """
    Parameters: 
        - `pred`: array of len 100 containing predictions
        - `base`: array of len 100 containing reference sentences
    """
    pred_labels = []
    base_labels = []
    for p, b in zip(pred_texts, base_texts):
        p = p.lower()
        b = b.lower()

        p_vec = []
        b_vec = []
        for lbl in label_cols:
            lbl_lc = lbl.lower()
            p_vec.append(1 if lbl_lc in p else 0)
            b_vec.append(1 if lbl_lc in b else 0)

        pred_labels.append(p_vec)
        base_labels.append(b_vec)

    rouge_scores = rouge.compute(predictions=pred_texts, references=base_texts)
    f1 = f1_score(pred_labels, base_labels,average='macro',zero_division=0)
    recall = recall_score(pred_labels, base_labels,average='micro',zero_division = 0 )
    precision = precision_score(pred_labels, base_labels,average='micro',zero_division = 0 )
    accuracy = accuracy_score(pred_labels, base_labels)

    return {
        "f1@100": f1,
        "recall@100": recall, 
        "accuracy@100": accuracy, 
        "precision@100": precision, 
        "rouge_scores@100": rouge_scores
    }


    
                
        
        
 


In [16]:

print(eval_model(pred_text, base_text))

print(f"predicted: {pred_text[7]}, base: {base_text[7]}")

{'f1@100': 0.24274703701639666, 'recall@100': 0.6265060240963856, 'accuracy@100': 0.33, 'precision@100': 0.6666666666666666, 'rouge_scores@100': {'rouge1': 0.9007042680502939, 'rouge2': 0.8385900647887502, 'rougeL': 0.8981917426043533, 'rougeLsum': 0.8982564370957373}}
predicted: This photo of a chest x-ray shows multiple findings including Infiltration and Pneumothorax. The image is taken from a AP view., base: This photo of a chest x-ray shows multiple findings including Consolidation and Infiltration. The image is taken from a AP view.


## Analysis of results and Possible Improvements 
As we can see our model is performing less then ideal. It has a really low accuracy in predicting the classes in our image embeddings. We can assume that the accuracy score of 39% obtained from the test comes from the fact that out dataset is hugely biased towards the No Findings label. 
We have different options in resolving this bias problem caused by our unbalanced Dataset. 
1 - Underscaling or Overscaling our dataset. Where either syntetic data is generated for the less prominent labels in order to balance out the distribution of the labels or we reduce the number of the samples belonging to the most prominent labels. Reducing might not be a good idea in our scenario of training a transformer because our dataset is already really small. So we will try to augmentate our dataset using augmentation tecniques. 
2 - Implementing a loss function that takes into account the frequency of the tokens present in our dataset so that the less frequent tokens get a higher weight in the loss calculation process. We will try 2 different algorithms: Focal Loss and Label Smoothing. 

### Focal Loss 
![](https://miro.medium.com/v2/resize:fit:1314/format:webp/1*0fiBC8RM_i1HBgXlil9Bkg.png)

p_t is the model’s predicted probability for the correct class, so if p_t is low (the model is uncertain), the scaling factor (1 - p_t)^\gamma will be large, making the loss higher for this sample. Essentially, this extra term controls the focus on misclassified samples.

### Step 1 - Calculate Token Weights 

The weight for each token will be given as max(freq(token))/freq(token)  

### Token Count Observations 
The alpha parameter push of 1120 might give a really strong boost to uncommon tokens causing an inbalance. Lets normalize the values. 

In [17]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

def min_max_scale(array):
    return (array - array.min()) / (array.max() - array.min())
    
def get_token_weights(dataloader, tokenizer,max_alpha_value = 10 ):
    counts = np.zeros(tokenizer.vocab_size, dtype=np.int64)
    for tokens, _, _ in dataloader:
        ids = tokens.cpu().numpy().ravel()
        ids = ids[ids < tokenizer.vocab_size]
        if ids.size > 0: 
            bc = np.bincount(ids, minlength=tokenizer.vocab_size) # form an array of length vocab where token_idx = idx on array and it contains the tot count of occurences
            counts += bc

    counts[0] = 0  #  token id 0 is padding ignore

    nonzero_indices = counts > 0
    weights = np.ones_like(counts, dtype=np.float64) 

    if np.any(nonzero_indices):
        max_count = counts[nonzero_indices].max()
        weights[nonzero_indices] = max_count / counts[nonzero_indices]

    
    
    

    weights = np.clip(weights, a_min=1.0, a_max=max_alpha_value)  # reduce the max value alpha_idx can have 

    weights[0] = 0.0 

    w = torch.from_numpy(weights).float().to(DEVICE)
    
    print(f"Token weight stats min={w.min().item()}, max={w.max().item()}, mean={w.mean().item()}")
    print(f"Number of non-padding tokens with non-zero counts: {np.sum(nonzero_indices)}")
    return w

In [18]:
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


class TokenFocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, ignore_index=0, reduction='mean'):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha            # a tensor of shape (V,) or None
        self.ignore_index = ignore_index
        self.reduction = reduction

    def forward(self, logits, targets):
        """
        logits: (B, L, V), targets: (B, L)
        """
        B, L, V = logits.shape
        logits = logits.reshape(-1, V)          # (B·L, V)
        targets = targets.flatten()             # (B·L,)
        log_p = F.log_softmax(logits, dim=1)    # (B·L, V)
        p = torch.exp(log_p)                    # (B·L, V)

        # standard CE
        ce = F.nll_loss(
            log_p, targets,
            weight=self.alpha,
            ignore_index=self.ignore_index,
            reduction='none'
        )                                        # (B·L,)

        # pick probability of the true class
        p_t = p[torch.arange(p.size(0)), targets]  # (B·L,)
        loss = ce * (1 - p_t) ** self.gamma        # focal weighting

        if self.reduction == 'mean':
            return loss.mean()
        else:
            return loss.sum()




def train_with_focal_loss(dataset: ClipCocoDataset, model: ClipCaptionModelTrain, args,
          lr: float = 2e-5, warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):

    device = torch.device('cuda:0')
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = model.to(device)
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    token_weights = get_token_weights(train_dataloader, tokenizer)
    num_training_steps = epochs * len(train_dataloader)
    scheduler = get_scheduler(
        name="linear",            # linear warmup & decay
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=num_training_steps,
    )
    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]
                        # ── before train loop ──
            # optional: pass the same `token_weight` from method ➊ as `alpha`
            focal_loss = TokenFocalLoss(gamma=2.0, alpha=token_weights, ignore_index=0)
            
            # ── in train loop ──
            loss = focal_loss(logits, tokens)
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            progress.set_postfix({"loss": loss.item()})
            progress.update()
            if (idx + 1) % 10000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        progress.close()
        if epoch % args.save_every == 0 or epoch == epochs - 1:
            torch.save(
                model.state_dict(),
                "./torch_model_with_loss.pt",
            )
    return model


def run_train_with_focal_loss(
    data: str = './train_data1.pkl',
    out_dir: str = './checkpoints',
    prefix: str = 'coco_prefix',
    epochs: int = 3,
    save_every: int = 1,
    prefix_length: int = 10,
    prefix_length_clip: int = 10,
    bs: int = 40,
    only_prefix: bool = False,
    mapping_type: str = 'mlp',
    num_layers: int = 8,
    is_rn: bool = False,
    normalize_prefix: bool = False,
):
    dataset = ClipCocoDataset(data, prefix_length, normalize_prefix=normalize_prefix)

    prefix_dim = 512

    # 3) Resolve mapping_type enum
    mapping_enum = {
        'mlp': MappingType.MLP,
        'transformer': MappingType.Transformer
    }[mapping_type]

    if only_prefix:
        model = ClipCaptionPrefix(
            prefix_length,
            clip_length=prefix_length_clip,
            prefix_size=prefix_dim,
            num_layers=num_layers,
            mapping_type=mapping_enum
        )
        print("Train only prefix")
    else:
        model = ClipCaptionModelTrain(
            prefix_length,
            clip_length=prefix_length_clip,
            prefix_size=prefix_dim,
            num_layers=num_layers,
            mapping_type=mapping_enum
        )
        print("Train both prefix and GPT")

    model = train_with_focal_loss(
        dataset,
        model,
        args=type('X', (), {
            'epochs': epochs,
            'save_every': save_every,
            'bs': bs
        })(),
        lr=2e-5,
        warmup_steps=5000,
        output_dir=out_dir,
        output_prefix=prefix
    )

    return model 


trained_model = run_train_with_focal_loss()

Data size is 64000
Train both prefix and GPT
Token weight stats min=0.0, max=10.0, mean=1.0090664625167847
Number of non-padding tokens with non-zero counts: 69
>>> Training epoch 0


coco_prefix: 100%|██████████| 1600/1600 [21:35<00:00,  1.23it/s, loss=0.154]


>>> Training epoch 1


coco_prefix: 100%|██████████| 1600/1600 [21:44<00:00,  1.23it/s, loss=0.228]


>>> Training epoch 2


coco_prefix: 100%|██████████| 1600/1600 [21:43<00:00,  1.23it/s, loss=0.134]


### Benchmark

In [19]:
base_text_focal = []
pred_text_focal = []

for _ , row in test_samples_100.iterrows(): 
    base_text_focal.append(row["Texts"])
    pred_text_focal.append(generate_caption(trained_model ,torch.FloatTensor(row["Image Features"])).replace(EOS_TOKEN, ""))

print("Base Loss:")
print(eval_model(pred_text_focal, base_text_focal))
print("-"*30)
print("Focal Loss:")
print(eval_model(pred_text_focal, base_text_focal))

Base Loss:
{'f1@100': 0.26743130154053224, 'recall@100': 0.6514522821576764, 'accuracy@100': 0.35, 'precision@100': 0.6709401709401709, 'rouge_scores@100': {'rouge1': 0.9005671530998016, 'rouge2': 0.839905009395042, 'rougeL': 0.8987563073727919, 'rougeLsum': 0.8989913821077967}}
------------------------------
Focal Loss:
{'f1@100': 0.26743130154053224, 'recall@100': 0.6514522821576764, 'accuracy@100': 0.35, 'precision@100': 0.6709401709401709, 'rouge_scores@100': {'rouge1': 0.9005671530998016, 'rouge2': 0.839905009395042, 'rougeL': 0.8987563073727919, 'rougeLsum': 0.8989913821077967}}


## Analysis of Training with Focal Loss